In [21]:
import random
import os
import pandas as pd 
from datetime import datetime
from tqdm import tqdm
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
import torch.nn as nn
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda', index=0)

In [22]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [23]:
train = pd.read_csv('../../DATA/train.csv')
test  = pd.read_csv('../../DATA/test.csv')

In [24]:
# 데이터 타입 변경, 열 이름 변경 

new_column_names = {
    'corporation': 'corp',
    'location': 'loc',
    'supply(kg)': 'supply',
    'price(원/kg)': 'price',
}

train = train.rename(columns=new_column_names)
test = test.rename(columns=new_column_names)

train['timestamp']  = pd.to_datetime(train['timestamp'])
test['timestamp']  = pd.to_datetime(test['timestamp'])

train['newitem'] = train['item'].str.cat([train['corp'], train['loc']], sep=' ')

In [25]:
#가격 시계열 price_data 생성하기 

def before_timeseries_data(train):
    unique_values = train['newitem'].unique()
    # 날짜 범위 생성
    start_date = datetime(2019, 1, 1).date()
    end_date =datetime(2023, 3, 3).date()
    dates = pd.date_range(start_date, end_date, freq='D')

    # 데이터프레임 생성
    price_data = pd.DataFrame(columns=['item'] + dates.strftime('%Y-%m-%d').tolist())
    price_data['item'] = unique_values

    # print(len(price_data)) 39 
    for i in range(len(price_data)) :
        price_data.iloc[i,1:] = train['price'][i*1523 : (i+1)*1523 ]
    price_data['item'] = price_data['item'].astype(str)
    price_data['corp'] = price_data['item'].str[3:4]

    price_data['loca'] = price_data['item'].str[-1:]
    price_data['product'] = price_data['item'].str[:2]
    price_data['product_loca'] =price_data['product'].str.cat(price_data['loca'], sep=' ')
    cols = price_data.columns[-4:]  # 가장 뒤의 3개 열의 열 이름을 선택
    price_data = price_data[cols.tolist() + price_data.columns[:-4].tolist()]  # 열 순서 변경
    
    return price_data

price_data = before_timeseries_data(train)

In [26]:


le = LabelEncoder()
price_data['product_loca'] = le.fit_transform(price_data['product_loca'])

/home/kiwoongyoon/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/kiwoongyoon/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [27]:
price_df = price_data.copy()

In [28]:
price_df

,corp,loca,product,product_loca,item,2019-01-01,2019-01-02,2019-01-03,2019-01-04,2019-01-05,...,2023-02-22,2023-02-23,2023-02-24,2023-02-25,2023-02-26,2023-02-27,2023-02-28,2023-03-01,2023-03-02,2023-03-03
0,A,J,TG,8,TG A J,0.0,0.0,1728.0,1408.0,1250.0,...,2813.0,2770.0,2633.0,3155.0,0.0,2657.0,3922.0,3397.0,3195.0,3640.0
1,A,S,TG,9,TG A S,0.0,0.0,2526.0,2134.0,2075.0,...,3444.0,3481.0,3518.0,4201.0,0.0,4166.0,4009.0,4173.0,4219.0,4089.0
2,B,J,TG,8,TG B J,0.0,0.0,1692.0,1516.0,1471.0,...,4175.0,6216.0,3558.0,2412.0,0.0,3540.0,3141.0,6382.0,3558.0,3470.0
3,B,S,TG,9,TG B S,0.0,0.0,1944.0,1815.0,1717.0,...,3202.0,3478.0,3939.0,3677.0,0.0,4057.0,3821.0,4037.0,4004.0,4241.0
4,C,J,TG,8,TG C J,0.0,0.0,1965.0,1794.0,1773.0,...,4017.0,4585.0,4835.0,5550.0,0.0,5037.0,2643.0,3742.0,3983.0,5175.0
5,C,S,TG,9,TG C S,0.0,0.0,2078.0,2002.0,1815.0,...,3228.0,3484.0,3773.0,4298.0,0.0,4180.0,4234.0,4357.0,4466.0,4748.0
6,D,J,TG,8,TG D J,0.0,0.0,1616.0,1337.0,1234.0,...,0.0,0.0,1838.0,1829.0,0.0,0.0,0.0,0.0,0.0,0.0
7,D,S,TG,9,TG D S,0.0,0.0,2048.0,1757.0,1719.0,...,2539.0,2955.0,3323.0,3321.0,0.0,3409.0,3236.0,4631.0,4114.0,4146.0
8,E,J,TG,8,TG E J,0.0,0.0,1692.0,1475.0,1392.0,...,3157.0,4793.0,2403.0,1489.0,0.0,2620.0,1673.0,0.0,1904.0,1622.0
9,E,S,TG,9,TG E S,0.0,0.0,2151.0,1829.0,2002.0,...,3400.0,3349.0,3026.0,2946.0,0.0,3418.0,3141.0,4235.0,3960.0,3791.0


In [43]:
class LTSF_NLinear(torch.nn.Module):
    def __init__(self, window_size, forecast_size, individual, feature_size):
        super(LTSF_NLinear, self).__init__()
        self.window_size = window_size
        self.forecast_size = forecast_size
        self.individual = individual
        self.channels = feature_size
        if self.individual:
            self.Linear = torch.nn.ModuleList()
            for i in range(self.channels):
                self.Linear.append(torch.nn.Linear(self.window_size, self.forecast_size))
        else:
            self.Linear = torch.nn.Linear(self.window_size, self.forecast_size)

    def forward(self, x):
        seq_last = x[:,-1:,:].detach()
        x = x - seq_last
        if self.individual:
            output = torch.zeros([x.size(0), self.forecast_size, x.size(2)],dtype=x.dtype).to(x.device)
            for i in range(self.channels):
                output[:,:,i] = self.Linear[i](x[:,:,i])
            x = output
        else:
            x = self.Linear(x.permute(0,2,1)).permute(0,2,1)
        x = x + seq_last
        return x

class Data(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

    def __len__(self):
        return len(self.Y)
    
    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]
    

In [44]:
# Function to reshape the data into a time series format for each ID
def reshape_data(df):
    time_series_data = []
    for idx, row in df.iterrows():
        sales_data = row[5:].values.astype(float)
        time_series_data.append(sales_data)
    return np.array(time_series_data)

# Modified time_slide_df function to work with the current data format
def time_slide_df(data, window_size, forecast_size):
    data_list = []
    dap_list = []
    for idx in range(0, len(data) - window_size - forecast_size + 1):
        x = data[idx:idx + window_size].reshape(window_size, 1)
        y = data[idx + window_size:idx + window_size + forecast_size]
        data_list.append(x)
        dap_list.append(y)
    return np.array(data_list, dtype='float32'), np.array(dap_list, dtype='float32')

# Function to create DataLoader for each ID
def create_dataloader(data, window_size, forecast_size, batch_size):
    X, Y = time_slide_df(data, window_size, forecast_size)
    print(f"{X.shape} is X shape , {Y.shape} is Y shape ")
    ds = Data(X, Y)
    return DataLoader(ds, batch_size=batch_size, shuffle=True)

# Reshape the data
time_series_data = reshape_data(price_df)

In [42]:
# Define the window size, forecast size, and batch size
window_size = 128
forecast_size = 28
batch_size = 2048
epoch_count = 777 #777
lr = 0.001
min_delta = 0.001
patience = 10

In [45]:
price_df

,corp,loca,product,product_loca,item,2019-01-01,2019-01-02,2019-01-03,2019-01-04,2019-01-05,...,2023-02-22,2023-02-23,2023-02-24,2023-02-25,2023-02-26,2023-02-27,2023-02-28,2023-03-01,2023-03-02,2023-03-03
0,A,J,TG,8,TG A J,0.0,0.0,1728.0,1408.0,1250.0,...,2813.0,2770.0,2633.0,3155.0,0.0,2657.0,3922.0,3397.0,3195.0,3640.0
1,A,S,TG,9,TG A S,0.0,0.0,2526.0,2134.0,2075.0,...,3444.0,3481.0,3518.0,4201.0,0.0,4166.0,4009.0,4173.0,4219.0,4089.0
2,B,J,TG,8,TG B J,0.0,0.0,1692.0,1516.0,1471.0,...,4175.0,6216.0,3558.0,2412.0,0.0,3540.0,3141.0,6382.0,3558.0,3470.0
3,B,S,TG,9,TG B S,0.0,0.0,1944.0,1815.0,1717.0,...,3202.0,3478.0,3939.0,3677.0,0.0,4057.0,3821.0,4037.0,4004.0,4241.0
4,C,J,TG,8,TG C J,0.0,0.0,1965.0,1794.0,1773.0,...,4017.0,4585.0,4835.0,5550.0,0.0,5037.0,2643.0,3742.0,3983.0,5175.0
5,C,S,TG,9,TG C S,0.0,0.0,2078.0,2002.0,1815.0,...,3228.0,3484.0,3773.0,4298.0,0.0,4180.0,4234.0,4357.0,4466.0,4748.0
6,D,J,TG,8,TG D J,0.0,0.0,1616.0,1337.0,1234.0,...,0.0,0.0,1838.0,1829.0,0.0,0.0,0.0,0.0,0.0,0.0
7,D,S,TG,9,TG D S,0.0,0.0,2048.0,1757.0,1719.0,...,2539.0,2955.0,3323.0,3321.0,0.0,3409.0,3236.0,4631.0,4114.0,4146.0
8,E,J,TG,8,TG E J,0.0,0.0,1692.0,1475.0,1392.0,...,3157.0,4793.0,2403.0,1489.0,0.0,2620.0,1673.0,0.0,1904.0,1622.0
9,E,S,TG,9,TG E S,0.0,0.0,2151.0,1829.0,2002.0,...,3400.0,3349.0,3026.0,2946.0,0.0,3418.0,3141.0,4235.0,3960.0,3791.0


In [53]:
for idx, (id_val, data) in tqdm(enumerate(zip(price_df["item"], time_series_data)), total=len(price_df["item"])):
    
    if idx ==0 :
        train_dl = create_dataloader(data, window_size, forecast_size, batch_size)
        print(len(data[-window_size: ]),(data[-window_size: ].shape) )

  0%|          | 0/39 [00:00<?, ?it/s]

(1368, 128, 1) is X shape , (1368, 28) is Y shape 
128 (128,)


# Model Train


In [15]:
# 3.001280905663967, 128, 2048, 777, 0.0015 이 경우가 제일 좋음 
def test (window_size,forecast_size, batch_size , epoch_count , lr, patience ): 
    future_predictions_by_id = {}
    loss_history = {}
    last_losses = []
    for idx, (id_val, data) in tqdm(enumerate(zip(price_df["item"], time_series_data)), total=len(price_df["item"])):
        mean_ = np.mean(data)
        std_ = np.std(data)
        print(data.shape) 
        if std_ == 0:
            standardized_data = data
        else:
            standardized_data = (data - mean_) / std_

        individual_loss_history = []
        # Create DataLoader
        train_dl = create_dataloader(standardized_data, window_size, forecast_size, batch_size)
        best_loss = float('inf')
        no_improvement_count = 0

        # Training the model
        DLinear_model = LTSF_NLinear(window_size=window_size, forecast_size=28, individual=False, feature_size=1)
        DLinear_model.to(device)
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(DLinear_model.parameters(), lr=lr)
        for epoch in range(1, epoch_count + 1):
            loss_list = []
            DLinear_model.train()
            for batch_idx, (data, target) in enumerate(train_dl):
                data, target = data.to(device), target.to(device)  # Move the data to the GPU if available
                optimizer.zero_grad()
                output = DLinear_model(data)
                loss = criterion(output, target.unsqueeze(-1))
                loss.backward()
                optimizer.step()
                loss_list.append(loss.item())
            if((epoch % 10) == 0):
                avg_loss = np.mean(loss_list)
                # print(f"Id {idx}, Epoch {epoch}: Loss = {avg_loss}, {id_val[:2]}")
                individual_loss_history.append(avg_loss)
                if avg_loss + min_delta < best_loss:
                    best_loss = avg_loss
                    no_improvement_count = 0
                else:
                    no_improvement_count += 1
                    if no_improvement_count >= patience:
                        # print(f"Early stopping at epoch {epoch} for ID {id_val} {idx} {avg_loss}")
                        weight = 0
                        weight_dict = { 
                                'TG' : 0.256,
                                'BC' : 0.23 ,
                                'RD' :0.205,
                                'CR' : 0.179,
                                'CB' : 0.128 
                                    }

                        weight = weight_dict[id_val[:2]] 

                        
                
                        last_losses.append(avg_loss*weight)
                        break


        loss_history[id_val] = individual_loss_history

        last_window_data = torch.tensor(standardized_data[-window_size:]).unsqueeze(0).unsqueeze(-1).float().to(device)
        future_prediction = DLinear_model(last_window_data)

        if std_ == 0:
            future_prediction = future_prediction.squeeze().detach().cpu().numpy()
        else:
            future_prediction = future_prediction.squeeze().detach().cpu().numpy() * std_ + mean_

        # Store the prediction
        future_predictions_by_id[id_val] = future_prediction

    # Future predictions for each ID from 2023-04-05 to 2023-04-25
    # future_predictions_by_id
    # print(sum(last_losses))
    print(f"window_size: {window_size}, batch_size:{batch_size} , epoch :{epoch_count}, lr:{lr}, patience: {patience}, loss:{sum(last_losses)}")
    return future_predictions_by_id

In [16]:
future_predictions_by_id= test(128,28 , 2048 , 777 , 0.0015, 10 )

  0%|          | 0/39 [00:00<?, ?it/s]

window_size: 128, batch_size:2048 , epoch :777, lr:0.0015, patience: 10, loss:3.1894988511204723


In [34]:
# bestloss = 100000
# bestwin, bestbatch, bestepoch, bestlr = 0,0,0,0
# for winsize in tqdm([64,72,84,112, 128 ]):
#     for batch_size in [256, 512,1024, 2048]:
#         for epoch_count in [ 777, 831]: 
#             for lr in [0.001, 0.0015]:
#                 onetestloss = test(winsize, 28,batch_size, epoch_count, lr, 10)
#                 if onetestloss < bestloss and onetestloss >=3 :
#                     bestloss,bestwin,bestbatch,bestepoch, bestlr =onetestloss,winsize,batch_size,epoch_count, lr
                    
# print(f"final : {bestloss,bestwin,bestbatch,bestepoch, bestlr}")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/39 [00:00<?, ?it/s]

window_size: 64, batch_size:256 , epoch :777, lr:0.001, patience: 10, loss:4.882851948005459


  0%|          | 0/39 [00:00<?, ?it/s]

window_size: 64, batch_size:256 , epoch :777, lr:0.0015, patience: 10, loss:4.890556828223169


...

window_size: 128, batch_size:2048 , epoch :777, lr:0.0015, patience: 10, loss:3.001280905663967


  0%|          | 0/39 [00:00<?, ?it/s]

window_size: 128, batch_size:2048 , epoch :831, lr:0.001, patience: 10, loss:1.6750190448164939


  0%|          | 0/39 [00:00<?, ?it/s]

window_size: 128, batch_size:2048 , epoch :831, lr:0.0015, patience: 10, loss:3.7966260848045352
final : (3.001280905663967, 128, 2048, 777, 0.0015)


In [17]:
submit = pd.read_csv('./../../DATA/sample_submission.csv')
submit

,ID,answer
0,TG_A_J_20230304,0
1,TG_A_J_20230305,0
2,TG_A_J_20230306,0
3,TG_A_J_20230307,0
4,TG_A_J_20230308,0
...,...,...
1087,RD_F_J_20230327,0
1088,RD_F_J_20230328,0
1089,RD_F_J_20230329,0
1090,RD_F_J_20230330,0


In [18]:
findata = []
for id_val, predictions in future_predictions_by_id.items():
    rounded_predictions = np.round(predictions).astype(np.float64)
    findata.extend(rounded_predictions)

submit['answer'] = findata
submit.loc[submit['answer'] <= 0, 'answer'] = 0
submit.loc[submit.index % 7 == 1, 'answer'] = 0

In [19]:
submit

,ID,answer
0,TG_A_J_20230304,2892.0
1,TG_A_J_20230305,0.0
2,TG_A_J_20230306,2691.0
3,TG_A_J_20230307,2751.0
4,TG_A_J_20230308,2590.0
...,...,...
1087,RD_F_J_20230327,442.0
1088,RD_F_J_20230328,416.0
1089,RD_F_J_20230329,443.0
1090,RD_F_J_20230330,453.0


In [20]:
submit.to_csv('../../DATA/SUBMIT/Nlinear4.csv', index=False)

In [25]:
submit2= pd.read_csv('./../../DATA/SUBMIT/Dlinear1.csv')
submit2

,ID,answer
0,TG_A_J_20230304,2750.0
1,TG_A_J_20230305,0.0
2,TG_A_J_20230306,2548.0
3,TG_A_J_20230307,2915.0
4,TG_A_J_20230308,2700.0
...,...,...
1087,RD_F_J_20230327,363.0
1088,RD_F_J_20230328,378.0
1089,RD_F_J_20230329,403.0
1090,RD_F_J_20230330,362.0


In [26]:
sub2 = submit2
sub2['answer'] = sub2['answer']+submit['answer']

In [28]:
sub2['answer'] = sub2['answer']/2

In [30]:
sub2.to_csv('../../DATA/SUBMIT/D+Llinear.csv', index=False)